In [22]:
from selenium import webdriver
import requests
import time
from time import sleep
import csv
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

import pickle
# How to pull all attributes from an xpath
# title = hover_spread.get_property('attributes')[0]

In [2]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

teams = []

driver = webdriver.Chrome('/Users/jonahmiller/Downloads/chromedriver 2')
driver.get("https://barttorvik.com/trank.php?year=2019#")

for i in range(1,104):
    team_location = "/html/body/div[1]/div/table/tbody/tr[{}]".format(i)
    elem = driver.find_element_by_xpath(team_location)
    team_name = elem.get_attribute('id')
    teams.append(team_name)
    

scrape_team = []
names = []
for i in range(len(teams)):
    if len(teams[i]) == 0:
        continue
    names.append(teams[i][1:])
check = teams

for i in range(len(names)):
    teams[i] = names[i]
    team = teams[i]
    if len(team) == 0:
        continue
    if '_' in teams[i]:
        teams[i] = teams[i].replace('_','+')
    if team == 'Saint_Mary_s':
        teams[i] = "Saint+Mary's"
    if team == 'St__John_s':
        teams[i] = "St+John's"
        

    scrape_team.append(teams[i])
    
# Clean up the end of the names 

for i in range(len(scrape_team)):
    last = scrape_team[i][-1]
    if last == '+':
        ins = scrape_team[i][:-1] + '.'
        scrape_team[i] = ins
    if scrape_team[i] == 'St++Bonaventure':
        scrape_team[i] = 'St.+Bonaventure'
        
        
vals = []
stats = []
total_team_data = {}
for i in range(len(scrape_team)):
    stats = []
    link = "https://barttorvik.com/team.php?team={}&year=2019".format(scrape_team[i])
    driver.get(link)
    time.sleep(0.2)
    vals.append(teams[i])
    record = '/html/body/div/div/h1/span'
    driver.find_element_by_xpath(record)
    for k in range(1,30):
        vals = [scrape_team[i]]
        hover_link = "/html/body/div/div/div[2]/div[4]/table/tbody/tr[{}]/td[8]".format(k)
        element_to_hover_over = driver.find_element_by_xpath(hover_link)
        hover = ActionChains(driver).move_to_element(element_to_hover_over)
        hover.perform()
        time.sleep(0.3)
        hover_spread = driver.find_element_by_xpath('/html/body/div[2]')
        Pregame_Spread = hover_spread.get_attribute('textContent')
        vals.append(Pregame_Spread)
        for j in range(1,31):
            path = "/html/body/div/div/div[2]/div[4]/table/tbody/tr[{}]/td[{}]".format(k,j)
            elem = driver.find_element_by_xpath(path)
            elem = elem.get_attribute('textContent')
            vals.append(elem)
        stats.append(vals)
    total_team_data[scrape_team[i]] = stats       
        
# THIS CODE WILL GRAB PREGAME SPREAD AND MAYBE PROJECTED SCORE... I THINK 
# element_to_hover_over = driver.find_element_by_xpath("/html/body/div/div/div[2]/div[4]/table/tbody/tr[1]/td[8]")
# hover = ActionChains(driver).move_to_element(element_to_hover_over)
# hover.perform()
# time.sleep(0.05)
# hover_spread = driver.find_element_by_xpath('/html/body/div[2]')
# Pregame_Spread = hover_spread.get_attribute('textContent')
# print(Pregame_Spread)


driver.close()


In [72]:
# This is a Solid starting point to work on the problem from, 
# Would be in my best interest to not overwrite this. 

# import pickle
# pickle_out = open("data.pickle", "wb")
# pickle.dump(total_team_data, pickle_out)
# pickle_out.close()


# team_pickle = open("teams.pickle", "wb")
# pickle.dump(scrape_team, team_pickle)
# team_pickle.close()

In [209]:
# for i in range(len(teams)):
#     name = teams[i]
#     if teams[i][-1] == '+':
#         teams[i] = teams[i][-1].replace('_','.')
#     print(teams[i])

In [326]:
# driver = webdriver.Chrome('/Users/jonahmiller/Downloads/chromedriver 2')
# for i in range(10):
#     vals = []
#     link = "https://barttorvik.com/team.php?team={}&year=2021".format(scrape_team[i])
#     driver.get(link)
#     vals.append(teams[i])
#     record = '/html/body/div/div/div[2]/div[4]/table/tbody/tr[1]'
#     driver.find_element_by_xpath(record)
#     rec = elem.get_attribute('textContent')
    
# driver.close()

In [1]:
# /html/body/div/div/div[2]/div[4]/table/tbody/tr[2]


# Here is where I do the bulk of the work.  Using the scraped list of top 100 teams I navigate to their respective 
# pages which contain 2020 - 2021 season with respective records and game stats.
# As the season continues this will continue to get larger and larger
# It would make sense to have a system set up to link this data to sql database and then can effeciently manipulate data

####  This was combined into one block at the top, left for future reference or in the event that I need to change something
# Future idea: Have a live feed of mathchups for the day and then automatically view algo odds 


# vals = []
# driver = webdriver.Chrome('/Users/jonahmiller/Downloads/chromedriver 2')
# stats = []
# total_team_data = {}
# for i in range(len(scrape_team)):
#     stats = []
#     link = "https://barttorvik.com/team.php?team={}&year=2021".format(scrape_team[i])
#     driver.get(link)
#     vals.append(teams[i])
#     record = '/html/body/div/div/h1/span'
#     driver.find_element_by_xpath(record)
#     for k in range(1,13):
#         vals = [scrape_team[i]]
#         for j in range(3,31):
#             path = "/html/body/div/div/div[2]/div[4]/table/tbody/tr[{}]/td[{}]".format(k,j)
#             elem = driver.find_element_by_xpath(path)
#             elem = elem.get_attribute('textContent')
#             vals.append(elem)
#         stats.append(vals)
#     total_team_data[scrape_team[i]] = stats
# driver.close()

In [221]:
# Here is a list of all the actual column names from the site.
# Need to actually walk through the scraping results to see what is pulled,
# There are most likely some inconsistencies. 

Cols = "‘Opponent’, ‘Result’, ‘Record’, ‘WAB’, ‘ADJO’, “ADJD’, ‘O - PPP’, ‘O - EFG%’, ‘O - TO%’, ‘O - OR%’, ‘O - FTR’, ‘O - 2P’, ‘O - 3P’,  ‘D - PPP’, ‘D - EFG%’, ‘D - TO%’, ‘D - OR%’, ‘D -  FTR’, ‘D -  2P’, ‘D - 3P’, ‘G-SC’, ‘+/-‘"
Cols = Cols.replace("‘", "'")

In [9]:
df = pd.DataFrame(total_team_data)  
df.to_csv('total_team_data.csv')